# Path e librerie

## Install

In [ ]:
from google.colab import drive
drive_dir = 'drive' ; drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 1.85 ms (started: 2021-11-15 20:24:35 +00:00)


## Import

In [ ]:
import tweepy
import numpy as np
from tweepy import OAuthHandler
from tqdm.notebook import tqdm
import pandas as pd
import io
import pprint
import os
import glob
import warnings
warnings.filterwarnings('ignore')
#ciao

time: 3.04 ms (started: 2021-11-15 21:12:03 +00:00)


In [ ]:
###add new cell

## Path

In [ ]:
##########################
### ROOT
##########################

prj_root_dir = f'/content/{drive_dir}/MyDrive/nucleare/'
os.makedirs(prj_root_dir, exist_ok=True)

##########################
### SCRAPING
##########################

scraping_root_dir = prj_root_dir + 'scraping/'
os.makedirs(scraping_root_dir, exist_ok=True)

##########################
### ETL
##########################

etl_root_dir = prj_root_dir + 'ETL/'
os.makedirs(etl_root_dir, exist_ok=True)

##########################
### DW
##########################

dw_root_dir = prj_root_dir + 'DW/'
os.makedirs(dw_root_dir, exist_ok=True)

time: 14.6 ms (started: 2021-11-15 21:12:04 +00:00)


# Ingest

## Codici Twitter

In [ ]:
# ciascuno metta le sue chiavi
consumer_key = 'cRKHyGGkxIcooSC2Gcq7KX7FW'
consumer_secret = 'XsOPBuoYcKe7qC3yAYbKPh8TRzL7fS92GxBXLAQkc7kt6d0hdO'
access_token = '1397138649284812800-NHIHjwNunVtPBLbk7Yt6OxwgMiq191'
access_secret = 'kypBtax3igYheMvXxDOi5prO4sIuiGYpmXHRguwQ3jM8t'
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

time: 4.24 ms (started: 2021-11-15 21:12:05 +00:00)


## Scraping dello storico

In [ ]:
####mettere il proprio nome
utente = 'natalia'

####mettere la lingua
lang = 'inglese'

In [ ]:
##############################################################################################################
##############################################################################################################
##############################################################################################################

#### mettere il numero del prossimo file number da cui partire. la prima volta 0.
#### ATTENZIONE, CONTROLLARE A CHE NUMERI SI è ARRIVATI ALTRIMENTI SI SOVRASCRIVE SEMPRE LO STESSO FILE!
file_number = 0
### mettere l'ultimo tweet id scaricato
last_id = -1
# mettere le proprie
query = 'nuclear'

##############################################################################################################
##############################################################################################################
##############################################################################################################

searched_tweets = []
last_id = -1
max_tweets = 100000
while len(searched_tweets) < max_tweets:
    count = max_tweets - len(searched_tweets)
    try:
        new_tweets = api.search(q=query, count=1000, max_id=str(last_id - 1), tweet_mode="extended")
        # If I've reached the end of the search then I'm done
        # immediately the cycle while
        if not new_tweets:
            break
        # added the data found to the list
        searched_tweets.extend(new_tweets)
        # retrieve the last id found
        last_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        print(e)
        break
len(searched_tweets)

KeyboardInterrupt: ignored

time: 2.03 s (started: 2021-11-15 21:29:32 +00:00)


In [ ]:
dict_ = {'id': [], 'user': [], 'date': [], 'full_text': [], 
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}

for status_j in searched_tweets[0:10000]:
    status = status_j._json
    dict_['id'].append(status['id'])
    dict_['user'].append(status['user']['screen_name'])
    dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
    dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
    dict_['location'].append(status['user']['location'])
    dict_['followers_count'].append(status['user']['followers_count'])
    dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
    dict_['date'].append(status['created_at'])
    dict_['full_text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['retweet_count'].append(status['retweet_count'])
    dict_['retweet'].append(status['retweeted'])
df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by=['date'], inplace=True, ascending=True)
df.head(10)

KeyError: ignored

time: 27.3 ms (started: 2021-11-15 21:12:19 +00:00)


In [ ]:
df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by=['date'], inplace=True, ascending=True)
df.head(10)

ValueError: ignored

time: 20.4 ms (started: 2021-11-15 21:13:03 +00:00)


In [ ]:
df.to_csv(f'{scraping_root_dir}_twitter_history_{utente}_{lang}_{file_number}.csv')

In [ ]:
df.count()

## Scraping dello stream

In [58]:
class MyStreamListener(tweepy.StreamListener):
    data = [] #qui ci metto i tweet che scarico man mano
    num = 0 #questo è un contatore settato a 0

    def store(self):
      dict_ = {'id': [], 'user': [], 'date': [], 'full_text': [], 
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}
      
      for status_j in self.data:
          status = status_j._json
          dict_['id'].append(status['id'])
          dict_['user'].append(status['user']['screen_name'])
          dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
          dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
          dict_['location'].append(status['user']['location'])
          dict_['followers_count'].append(status['user']['followers_count'])
          dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
          dict_['date'].append(status['created_at'])
          dict_['full_text'].append(status['full_text'])
          dict_['favorite_count'].append(status['favorite_count'])
          dict_['retweet_count'].append(status['retweet_count'])
          dict_['retweet'].append(status['retweeted'])

      df = pd.DataFrame.from_dict(dict_, orient="columns")
      df = df.set_index("id")
      df.sort_values(by='favorite_count', inplace=True, ascending=False)
      df.to_csv(f'{scraping_root_dir}_twitter_stream_{utente}_{lang}_' + str(self.num) + '.csv') #salvo in csv
      self.num = self.num + 1
      self.data = []


    def on_status(self, status):
        self.data.append(status)
        # to pandas
        # store su google drive
        if len(self.data) > 5000:
          self.store()

time: 37.9 ms (started: 2021-11-15 22:17:42 +00:00)


In [59]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener, tweet_mode="extended")

time: 2.06 ms (started: 2021-11-15 22:17:45 +00:00)


In [60]:
query = ['energytransaction' or 'nuclear']
myStream.filter(track=query)

KeyboardInterrupt: ignored

time: 4.37 s (started: 2021-11-15 22:17:51 +00:00)


In [62]:
len(myStreamListener.data)

0

time: 4.37 ms (started: 2021-11-15 22:18:02 +00:00)


In [ ]:
dict_ = {'id': [], 'user': [], 'date': [], 'full_text': [], 
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}
#
for status_j in myStreamListener.data:
    status = status_j._json
    dict_['id'].append(status['id'])
    dict_['user'].append(status['user']['screen_name'])
    dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
    dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
    dict_['location'].append(status['user']['location'])
    dict_['followers_count'].append(status['user']['followers_count'])
    dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['full_text'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['retweet_count'].append(status['retweet_count'])
    dict_['retweet'].append(status['retweeted'])

In [ ]:
df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by='favorite_count', inplace=True, ascending=False)
df.head(10)

In [ ]:
df.to_csv(f'{scraping_root_dir}_twitter_stream_{utente}__{lang}_end.csv') #scarico l'ultimo stream

In [ ]:
#unisco tutti i csv

path = scraping_root_dir

all_files = glob.glob(os.path.join(path, "twitter_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged   = pd.concat(df_from_each_file, ignore_index=True)
df_merged.to_csv(f'{etl_root_dir}twitter_merge.csv')

# ETL

# Sentiment

# BoW, Word embeddings, Lime?

# SocialNetwork Analisys